In [1]:
import sys
import pandas as pd

sys.path.append("..")
import os
import rbf_functions

In [2]:
rbfs = [
    rbf_functions.original_rbf,
    rbf_functions.squared_exponential_rbf,
    rbf_functions.inverse_quadratic_rbf,
    rbf_functions.inverse_multiquadric_rbf,
    rbf_functions.exponential_rbf,
    rbf_functions.matern32_rbf,
    rbf_functions.matern52_rbf,
]

# Load data

In [3]:
gddata = {}
eidata = {}
hvdata = {}
varlist = ["10", "20", "30", "40", "50", "60", "70", "80", "90", "100"]
for entry in rbfs:
    rbf = entry.__name__
    df_gd = pd.read_csv(f"./gd/{rbf}_gd_all.csv")
    df_ei = pd.read_csv(f"./epind/{rbf}_ei_all.csv")
    df_hv = pd.read_csv(f"./hv/{rbf}_hv_global.csv")
    varlist = ["10", "20", "30", "40", "50", "60", "70", "80", "90", "100"]
    gddata[rbf] = {}
    eidata[rbf] = {}
    hvdata[rbf] = {}
    for i in varlist:  # add metric value at 100.000 nfe to dict for each seed
        gddata[rbf][i] = df_gd[i].dropna().iloc[-1]
        eidata[rbf][i] = df_ei[i].dropna().iloc[-1]
        hvdata[rbf][i] = df_hv[i].dropna().iloc[-1]
df_gd_m = pd.DataFrame.from_dict(gddata, orient="index")
df_ei_m = pd.DataFrame.from_dict(eidata, orient="index")
df_hv_m = pd.DataFrame.from_dict(hvdata, orient="index")

In [4]:
global_hv = pd.read_csv("./hv/hypervolume_global_refset.csv", index_col="Unnamed: 0")
global_hv.iloc[0, 0]

0.4148666686740452

In [5]:
df_hv_m /= global_hv.iloc[0, 0]
df_hv_m

,10,20,30,40,50,60,70,80,90,100
original_rbf,0.473642,0.460548,0.469249,0.464611,0.466293,0.469506,0.469849,0.469330,0.468911,0.470459
squared_exponential_rbf,0.469794,0.475615,0.466652,0.467731,0.468423,0.466859,0.477527,0.476514,0.477385,0.468756
inverse_quadratic_rbf,0.464304,0.461733,0.463739,0.462276,0.467253,0.463917,0.467115,0.463981,0.468205,0.461914
inverse_multiquadric_rbf,0.461735,0.463499,0.466708,0.463757,0.464776,0.463513,0.462911,0.461405,0.462958,0.463490
exponential_rbf,0.464702,0.470920,0.463505,0.469927,0.464800,0.467193,0.464511,0.464822,0.462464,0.462309
matern32_rbf,0.469204,0.466453,0.469122,0.465585,0.466222,0.465666,0.466151,0.470336,0.466076,0.471932
matern52_rbf,0.465643,0.466720,0.463463,0.473010,0.471592,0.464313,0.465671,0.463376,0.466856,0.466108


In [6]:
import os

paretosets = {}
counts = {}
for entry in rbfs:
    name = entry.__name__
    # load variables
    varlist = []
    variables = []

    output_dir = os.path.abspath("./refsets/")

    for filename in os.listdir(output_dir):
        if filename == f"{name}_refset.csv":
            varlist.append(filename[:-4])
            df_vars = pd.read_csv(os.path.join(output_dir, filename))
            counts[name] = len(df_vars)
            paretosets[name] = df_vars
    variables = df_vars.values

counts

{'original_rbf': 21,
 'squared_exponential_rbf': 23,
 'inverse_quadratic_rbf': 19,
 'inverse_multiquadric_rbf': 19,
 'exponential_rbf': 23,
 'matern32_rbf': 20,
 'matern52_rbf': 22}

In [7]:
df_sol = pd.DataFrame.from_dict(counts, orient="index")
df_sol

,0
original_rbf,21
squared_exponential_rbf,23
inverse_quadratic_rbf,19
inverse_multiquadric_rbf,19
exponential_rbf,23
matern32_rbf,20
matern52_rbf,22


In [8]:
df_table = pd.DataFrame()
df_table["solutions"] = df_sol[0]
df_table["Generational Distance avg"] = df_gd_m.T.mean()
df_table["Epsilon Indicator avg"] = df_ei_m.T.mean()
df_table["Hypervolume avg"] = df_hv_m.T.mean()
df_table

,solutions,Generational Distance avg,Epsilon Indicator avg,Hypervolume avg
original_rbf,21,0.018994,0.242975,0.468240
squared_exponential_rbf,23,0.020191,0.253429,0.471526
inverse_quadratic_rbf,19,0.022929,0.271876,0.464444
inverse_multiquadric_rbf,19,0.025915,0.259099,0.463475
exponential_rbf,23,0.021459,0.285110,0.465515
matern32_rbf,20,0.015791,0.318811,0.467675
matern52_rbf,22,0.020485,0.268411,0.466675


In [9]:
import collections

df_global = pd.read_csv(f"./refsets/all_rbf_refset.csv")
global_refset = set([tuple(entry) for entry in df_global.values.tolist()])

contribution = collections.defaultdict(int)
for rbf in paretosets:
    for solution in paretosets[rbf].values.tolist():
        solution = tuple(solution)
        if solution in global_refset:
            contribution[rbf] += 1

contribution = {k: round(v / len(global_refset), 3) for k, v in contribution.items()}

In [10]:
contribution

{'original_rbf': 0.25,
 'squared_exponential_rbf': 0.542,
 'inverse_multiquadric_rbf': 0.042,
 'exponential_rbf': 0.083,
 'matern32_rbf': 0.083}

In [11]:
df_table["set contribution"] = pd.Series(contribution)

In [12]:
df_table

,solutions,Generational Distance avg,Epsilon Indicator avg,Hypervolume avg,set contribution
original_rbf,21,0.018994,0.242975,0.468240,0.250
squared_exponential_rbf,23,0.020191,0.253429,0.471526,0.542
inverse_quadratic_rbf,19,0.022929,0.271876,0.464444,NaN
inverse_multiquadric_rbf,19,0.025915,0.259099,0.463475,0.042
exponential_rbf,23,0.021459,0.285110,0.465515,0.083
matern32_rbf,20,0.015791,0.318811,0.467675,0.083
matern52_rbf,22,0.020485,0.268411,0.466675,NaN


In [13]:
df_table.to_latex()

/var/folders/_t/l6cr0r0d1m108xdrmrtz7stsb_csl0/T/ipykernel_91002/909359497.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_table.to_latex()


'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &  solutions &  Generational Distance avg &  Epsilon Indicator avg &  Hypervolume avg &  set contribution \\\\\n\\midrule\noriginal\\_rbf             &         21 &                   0.018994 &               0.242975 &         0.468240 &             0.250 \\\\\nsquared\\_exponential\\_rbf  &         23 &                   0.020191 &               0.253429 &         0.471526 &             0.542 \\\\\ninverse\\_quadratic\\_rbf    &         19 &                   0.022929 &               0.271876 &         0.464444 &               NaN \\\\\ninverse\\_multiquadric\\_rbf &         19 &                   0.025915 &               0.259099 &         0.463475 &             0.042 \\\\\nexponential\\_rbf          &         23 &                   0.021459 &               0.285110 &         0.465515 &             0.083 \\\\\nmatern32\\_rbf             &         20 &                   0.015791 &               0.318811 &         0.467675 &             0.083 \\